#### Module Importation

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

%reload_ext autoreload
%autoreload 
from HelperFunctions import missingValuesInfo

from sklearn.impute import SimpleImputer
from fancyimpute import KNN
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

Using TensorFlow backend.


#### Helper Function

In [4]:
def imputer_score(imputer,estimator,x_miss,y):
    x_imputed=imputer.fit_transform(x_miss)
    impute_scores=cross_val_score(estimator,x_imputed,y,scoring='neg_mean_squared_error',cv=5)
    return -1*impute_scores.mean()

# Import Data

In [5]:
df=pd.read_csv('lean_df_4.csv',index_col='id')
mybatch=minibatch(df)

/home/auscheng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (34,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/auscheng/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


# Variable Type Definition

In [7]:
df_number = df.select_dtypes(include = 'number')
df_object = df.select_dtypes(include = 'object')
df_category = df.select_dtypes(include = 'category')
df_boolean = df.select_dtypes(include = 'bool')
df_datetime = df.select_dtypes(include = 'datetime')
df_timedelta = df.select_dtypes(include = 'timedelta')
#######################################################
nominal_var=list(df_object.columns)
ordinal_var=list(df_number.columns)
continuous_var=list(df_number.columns)
time_var=list(df_datetime.columns)

# Instantiate Imputer 

In [8]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_median = SimpleImputer(missing_values=np.nan, strategy= 'median')
imp_mode = SimpleImputer(missing_values=np.nan,strategy="most_frequent")
imp_const = SimpleImputer(missing_values=np.nan,fill_value='none',strategy="constant")
imp_const_n1 = SimpleImputer(missing_values=np.nan,fill_value=-1,strategy="constant")
imp_KNN = KNN(k=3)
# imp_MICE = IterativeImputer(random_state=0)


In [9]:
df[continuous_var]=imp_mean.fit_transform(df[continuous_var])
df[ordinal_var]=imp_mean.fit_transform(df[ordinal_var])
df[nominal_var]=imp_const_n1.fit_transform(df[nominal_var])
df[continuous_var]=imp_KNN.fit_transform(df[continuous_var])
df[continuous_var]=imp_MICE.fit_transform(df[continuous_var])

/home/auscheng/anaconda3/lib/python3.7/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


MemoryError: 

In [ ]:
x=df_number.drop('SalePrice',axis=1)
y=df_number.SalePrice
regressor=RandomForestRegressor(n_estimators=100,random_state=0)
classifier=RandomForestClassifier(n_estimators=100,random_state=0)
imputer_score(imp_MICE,regressor,x,y)

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
    ]
for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))

In [38]:
from sklearn.impute import MissingIndicator
x=np.array([[np.nan, 1, 3],[4, 0, np.nan],[8, 1, 0]])
indicator = MissingIndicator()
indicator.fit_transform(x)  

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
import numpy as np
>>> from sklearn.impute import MissingIndicator
>>> X1 = np.array([[np.nan, 1, 3],
...                [4, 0, np.nan],
...                [8, 1, 0]])
>>> X2 = np.array([[5, 1, np.nan],
...                [np.nan, 2, 3],
...                [2, 4, 0]])
>>> indicator = MissingIndicator()
>>> indicator.fit(X1)  